# Week Seven Class Two: Fixed Cost Example 7.2

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Model Definition](#3)<br>
4. [Model Solution](#4)<br>

## 1. Problem Statement

Here we will solve the problem from example 7.2 in the textbook showing the hours required for 3 machines, the 3 potential product families to invest, a fixed sales cost, and a demand ceiling for each product, and a variable profit per unit. The data are in `w07-c02-linking-cons.xlsx`.

##### [Back to Top](#Top)

## 2. Data

In [1]:
import pandas as pd
import pyomo.environ as pe
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
raw_data = pd.read_excel('w07-c02-linking-cons.xlsx', sheet_name = 'Example 7.2')
raw_data

,Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Hours Required/1000 Units,F1,F2,F3,Hours Available,NaN,NaN
2,NaN,Dept A,3,4,8,2000,NaN,NaN
3,NaN,Dept B,3,5,6,2000,NaN,NaN
4,NaN,Dept C,2,3,9,2000,NaN,NaN
5,NaN,Demand ceiling (000s),300,200,50,NaN,NaN,NaN
6,NaN,Profit/unit ($),1.2,1.8,2.2,NaN,NaN,NaN
7,NaN,Sales cost ($000),60,200,100,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Decisions,NaN,F1,F2,F3,NaN,NaN,NaN


First we will capture the coefficients which will the the price per unit, the sales fixed cost, and the hours for each department needed for each product family.

In [3]:
DV_indexes = ['F1', 'F2', 'F3']
coef = pd.DataFrame(raw_data.iloc[[6, 7, 2, 3, 4], [2, 3, 4]])
coef.index = ['price', 'cost', 'DeptA', 'DeptB', 'DeptC']
coef.columns = DV_indexes
coef

,F1,F2,F3
price,1.2,1.8,2.2
cost,60,200,100
DeptA,3,4,8
DeptB,3,5,6
DeptC,2,3,9


The next tables will give us our max values for our constraints - we have hours capacity for each department and we have a set demand for each product family.

In [4]:
hours = pd.DataFrame(raw_data.iloc[2:5, 5])
hours.columns = ['hours']
hours.index = coef.index[2:]
hours

,hours
DeptA,2000
DeptB,2000
DeptC,2000


In [5]:
demand = pd.DataFrame()
demand.loc['F1','demand'] = raw_data.iloc[5, 2]
demand.loc['F2','demand'] = raw_data.iloc[5, 3]
demand.loc['F3','demand'] = raw_data.iloc[5, 4]

demand

,demand
F1,300.0
F2,200.0
F3,50.0


##### [Back to Top](#Top)

## 3. Model Definition

Now let's solve the model.

In [6]:
model = pe.ConcreteModel()

### Define Decision Variables

__NOTE:__ for this model you have 2 sets of decision variables: (1) `F` variables will cover the 3 product families and (2) `y` are binary utilization variables used to create the fixed cost linking constraints.

In [7]:
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.y = pe.Var(DV_indexes, domain = pe.Binary)
model.x.pprint()
model.y.pprint()

x : Size=3, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     F1 :     0 :  None :  None : False :  True : NonNegativeReals
     F2 :     0 :  None :  None : False :  True : NonNegativeReals
     F3 :     0 :  None :  None : False :  True : NonNegativeReals
y : Size=3, Index=y_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     F1 :     0 :  None :     1 : False :  True : Binary
     F2 :     0 :  None :     1 : False :  True : Binary
     F3 :     0 :  None :     1 : False :  True : Binary


### Define objective function

Note that this will be the total profit which is calculated from the fixed price and variable sales costs. Make sure you see how these are calculated in the Excel sheet before you try to implement here.

In [8]:
#obj funct max price * x - cost * y
model.obj = pe.Objective(expr = sum([coef.loc['price', index]*model.x[index] for index in DV_indexes])
                         - sum([coef.loc['cost', index]*model.y[index] for index in DV_indexes]),
                         sense = pe.maximize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 1.2*x[F1] + 1.8*x[F2] + 2.2*x[F3] - (60*y[F1] + 200*y[F2] + 100*y[F3])


### Define Constraints

In [9]:
#Hours Capacity Constraints
model.DeptA = pe.Constraint(expr = sum([coef.loc['DeptA', index]*model.x[index] 
                                       for index in DV_indexes]) <= hours.loc['DeptA', 'hours'])
model.DeptB = pe.Constraint(expr = sum([coef.loc['DeptB', index]*model.x[index] 
                                       for index in DV_indexes]) <= hours.loc['DeptB', 'hours'])
model.DeptC = pe.Constraint(expr = sum([coef.loc['DeptC', index]*model.x[index] 
                                       for index in DV_indexes]) <= hours.loc['DeptC', 'hours'])
#Demand and Linking Constraints
model.LinkF1 = pe.Constraint(expr=model.x['F1'] <= demand.loc['F1', 'demand'] * model.y['F1'])
model.LinkF2 = pe.Constraint(expr=model.x['F2'] <= demand.loc['F2', 'demand'] * model.y['F2'])
model.LinkF3 = pe.Constraint(expr=model.x['F3'] <= demand.loc['F3', 'demand'] * model.y['F3'])

In [10]:
for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

DeptA : Size=1, Index=None, Active=True
    Key  : Lower : Body                        : Upper  : Active
    None :  -Inf : 3*x[F1] + 4*x[F2] + 8*x[F3] : 2000.0 :   True
DeptA None
DeptB : Size=1, Index=None, Active=True
    Key  : Lower : Body                        : Upper  : Active
    None :  -Inf : 3*x[F1] + 5*x[F2] + 6*x[F3] : 2000.0 :   True
DeptB None
DeptC : Size=1, Index=None, Active=True
    Key  : Lower : Body                        : Upper  : Active
    None :  -Inf : 2*x[F1] + 3*x[F2] + 9*x[F3] : 2000.0 :   True
DeptC None
LinkF1 : Size=1, Index=None, Active=True
    Key  : Lower : Body                : Upper : Active
    None :  -Inf : x[F1] - 300.0*y[F1] :   0.0 :   True
LinkF1 None
LinkF2 : Size=1, Index=None, Active=True
    Key  : Lower : Body                : Upper : Active
    None :  -Inf : x[F2] - 200.0*y[F2] :   0.0 :   True
LinkF2 None
LinkF3 : Size=1, Index=None, Active=True
    Key  : Lower : Body               : Upper : Active
    None :  -Inf : x[F3] - 50.0

##### [Back to Top](#Top)

## 4. Model Solution

In [11]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

ok optimal


### Optimal Objective Value

In [12]:
obj_val = model.obj.expr()
print(f'optimal objective value maximum profit = ${obj_val:.2f}')

optimal objective value maximum profit = $460.00


### Optimal Decision Variables

In [13]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

,F1,F2,F3
x,300.0,200.0,0.0
y,1.0,1.0,0.0
